In [1]:
import pandas as pd
import re

In [4]:
link_to_drive = "C:/Users/erika/OneDrive/Desktop/" 
file_path = "Merged_Processed_Data - 10-19-23.csv"
files_toclean = pd.read_csv(link_to_drive + file_path)
files_toclean.set_index('Calibrated m/z', inplace=True)
# Sort/reorder by index
files_toclean = files_toclean.sort_index()
abspres = (files_toclean != 0).astype(int)
out_file = 'C:/Users/erika/OneDrive/Desktop/Merged_Processed_Data - 10-19-23_ECF.csv'

In [5]:
"""
select_columns_containing: Returns columns containing the specified substring.
remove_columns_and_report: Removes columns based on the 'country.year' pattern, but retains those with the 'dil' keyword. Also, provides a report on how many columns contain "dil" and lists the columns that were removed.
remove_pattern1: Removes the pattern \d{5}\.corems from a string.
remove_after_dil_dill: Removes everything after "dil" or "dill" in column names.
remove_manyfiles_prefix: Removes the "ManyFiles_" prefix from column names.
remove_strings_from_segments: Removes segments from column names that start with specific keywords.
adjust_column_names: Modifies column names to replace '.' with '_' and ensures an underscore after the first string segment.
trim_columns_after_first_number: Trims the column names after the first number.
replace_sweden_with_swed: Replaces the occurrence of "Sweden" with "Swed" in column names."""

'\nselect_columns_containing: Returns columns containing the specified substring.\nremove_columns_and_report: Removes columns based on the \'country.year\' pattern, but retains those with the \'dil\' keyword. Also, provides a report on how many columns contain "dil" and lists the columns that were removed.\nremove_pattern1: Removes the pattern \\d{5}\\.corems from a string.\nremove_after_dil_dill: Removes everything after "dil" or "dill" in column names.\nremove_manyfiles_prefix: Removes the "ManyFiles_" prefix from column names.\nremove_strings_from_segments: Removes segments from column names that start with specific keywords.\nadjust_column_names: Modifies column names to replace \'.\' with \'_\' and ensures an underscore after the first string segment.\ntrim_columns_after_first_number: Trims the column names after the first number.\nreplace_sweden_with_swed: Replaces the occurrence of "Sweden" with "Swed" in column names.'

In [6]:
def select_columns_containing(df, substring):
    """
    Select columns that contain the specified substring.

    Parameters:
    - df: DataFrame to be filtered.
    - substring: The string to look for in column names.

    Returns:
    - DataFrame containing only the columns with the specified substring in their name.
    """
    return df[[col for col in df.columns if substring in col]]

def remove_columns_and_report(df):
    """
    Remove columns based on the 'country.year' pattern, but keep those with 'dil' keyword.
    Produce a report on how many columns contain "dil" and list the columns that were removed.

    Parameters:
    - df: DataFrame to be filtered.

    Returns:
    - DataFrame excluding the undesired columns.
    """

    # Use regular expressions to identify the country and year pattern
    pattern = re.compile(r'([A-Za-z]+)\.(\d{2})')

    # Find the country.year components and store them
    matches = [pattern.search(col) for col in df.columns]
    display(matches)
    country_year_components = [match.group(0) for match in matches if match]

    # Identify the unique country.year components
    unique_components = set(country_year_components)

    # Filter columns
    columns_to_remove = []

    for component in unique_components:
        component_cols = [col for col in df.columns if component in col]
        # Check if any of the columns for this component contains 'dil'
        has_dil = any(['dil' in col for col in component_cols])
        if has_dil:
            columns_to_remove.extend([col for col in component_cols if 'dil' not in col])

    # Remove the identified columns
    df_cleaned = df.drop(columns=columns_to_remove)

    # Report
    dil_columns = [col for col in df.columns if 'dil' in col]
    print(f"Total columns containing 'dil': {len(dil_columns)}")
    print(f"Columns removed: {', '.join(columns_to_remove)}")

    return df_cleaned

def remove_pattern1(column_name):
    return re.sub(r'\d{5}\.corems', '', column_name)

def prefix_dil_if_present(df):
    """
    Add "dil_" to the beginning of column names if they contain "dil".

    Parameters:
    - df: DataFrame to modify.

    Returns:
    - Modified DataFrame with updated column names.
    """

    def prefix_dil(col):
        if 'dil' in col:
            return 'retain_' + col
        else:
            return col

    df.columns = [prefix_dil(col) for col in df.columns]
    return df

def remove_after_dil_dill(df):
    """
    Remove strings that come after "dil" or "dill" in column names of a DataFrame.

    Parameters:
    - df: DataFrame to modify.

    Returns:
    - Modified DataFrame with updated column names.
    """

    def clean_column_name(col):
        if 'dil' in col:
            return col.split('dil', 1)[0]
        else:
            return col

    df.columns = [clean_column_name(col) for col in df.columns]
    return df

def remove_manyfiles_prefix(df):
    """
    Remove the "ManyFiles_" prefix from column names of a DataFrame.

    Parameters:
    - df: DataFrame to modify.

    Returns:
    - Modified DataFrame with updated column names.
    """

    df.columns = [col.replace('ManyFiles_', '') for col in df.columns]
    return df

def remove_strings_from_segments(df):
    """
    Remove segments of column names that start with specific strings.

    Parameters:
    - df: DataFrame to modify.

    Returns:
    - Modified DataFrame with updated column names.
    """

    unwanted_starts = ["MGC", "FTMS", "Lakes", "FJ"]

    def replace_unwanted_segments(col):
        segments = col.split('_')
        filtered_segments = [segment for segment in segments if not any(segment.startswith(unwanted) for unwanted in unwanted_starts)]
        return '_'.join(filtered_segments)

    df.columns = [replace_unwanted_segments(col) for col in df.columns]
    return df


def adjust_column_names(df):
    """
    Adjust the column names based on the criteria:
    1. Replace a '.' with '_'.
    2. Insert an underscore after the first string segment if it's not already present.

    Parameters:
    - df: DataFrame to modify.

    Returns:
    - Modified DataFrame with updated column names.
    """

    def process_colname(col):
        col = col.replace('.', '_')  # Replace '.' with '_'

        # Add underscore after the first string if it's not already present
        match = re.match(r'^([a-zA-Z]+)(?![a-zA-Z_])', col)
        if match:
            prefix = match.group(1)
            col = col.replace(prefix, prefix + "_", 1)  # replace only the first occurrence

        return col

    df.columns = [process_colname(col) for col in df.columns]
    return df

def trim_columns_after_first_number(df):
    """
    Trims the column names of a DataFrame after the first number.

    Parameters:
    - df: Input DataFrame.

    Returns:
    - DataFrame with modified column names.
    """
    def trim_after_first_number(col_name):
        match = re.match(r'^([a-zA-Z_]+?)(\d+)', col_name)
        if match:
            return match.group(1) + match.group(2)
        else:
            return col_name

    df.columns = [trim_after_first_number(col) for col in df.columns]
    return df

def replace_sweden_with_swed(df):
    """
    Replaces the occurrence of "Sweden" with "Swed" in column names.

    Parameters:
    - df: Input DataFrame.

    Returns:
    - DataFrame with modified column names.
    """
    df.columns = [col.replace('Sweden', 'Swed') for col in df.columns]
    return df

def drop_matching_columns_without_prefix(df, prefix="retain_"):
    """
    Drop columns that have a matching counterpart with the specified prefix.

    Parameters:
    - df: DataFrame to be filtered.
    - prefix: The prefix to look for in column names.

    Returns:
    - DataFrame without the columns that have a matching counterpart with the prefix.
    """

    # Identify columns with the prefix
    prefixed_cols = [col for col in df.columns if col.startswith(prefix)]

    # Identify the matching columns without the prefix
    matching_cols_to_drop = [col.replace(prefix, "") for col in prefixed_cols if col.replace(prefix, "") in df.columns]

    return df.drop(columns=matching_cols_to_drop)


def remove_retain_prefix(df):
    """
    Remove the "retain_" prefix from column names of a DataFrame.

    Parameters:
    - df: DataFrame to modify.

    Returns:
    - Modified DataFrame with updated column names.
    """

    df.columns = [col.replace('retain_', '') for col in df.columns]
    return df

def remove_columns_with_substring(df, substring):
    """Remove all columns containing a specific substring."""
    cols_to_remove = [col for col in df.columns if substring in col]
    return df.drop(columns=cols_to_remove)


def merge_and_export(df_original, df_cleaned, filename):
    """Merge two DataFrames and export to CSV."""
    # Remove 'ManyFiles' columns
    df_original = remove_columns_with_substring(df_original, 'ManyFiles')
    # Merge with cleaned columns
    df_combined = pd.concat([df_original, df_cleaned], axis=1)
    # Export to CSV
    df_combined.to_csv(filename, index=False)
    return df_combined

def clean_the_names(df):
    filtered_df = select_columns_containing(df, 'ManyFiles') #290 samples only with J's sample codes
    filtered_df.columns = [remove_pattern1(col) for col in filtered_df.columns]
    dil = prefix_dil_if_present(filtered_df)
    no_dil = remove_after_dil_dill(dil)
    nomanyfiles = remove_manyfiles_prefix(no_dil)
    no_MGC = remove_strings_from_segments(nomanyfiles)
    no_MGC_segment = adjust_column_names(no_MGC)
    df = pd.DataFrame(no_MGC_segment)
    df = trim_columns_after_first_number(df)
    newdf = replace_sweden_with_swed(df)
    second = drop_matching_columns_without_prefix(newdf)
    return second 

def combine_columns(df):
    """
    Combine columns with the same name.
    After combining, if any of the columns has a 1, set the row value to 1, otherwise 0.

    Parameters:
    - df: DataFrame to modify.

    Returns:
    - Modified DataFrame with combined columns.
    """

    # Get unique column names
    unique_cols = df.columns.unique()

    for col in unique_cols:
        # If there are multiple columns with the same name
        if sum(df.columns == col) > 1:
            # Combine columns and check if any of them has a 1
            df[col] = df[col].max(axis=1).astype(int)

            # Drop duplicate columns, keeping only the first occurrence
            df = df.loc[:, ~df.columns.duplicated(keep='first')]

    return df

In [7]:
cleaned_abs_pres= clean_the_names(abspres)
cleaned_relInt= clean_the_names(files_toclean)
DetectedinAll = cleaned_abs_pres.multiply(cleaned_relInt)

In [8]:
result = DetectedinAll.groupby(by=DetectedinAll.columns, axis=1).mean()
cleaned_result = combine_columns(result)
df_cleaned = remove_retain_prefix(cleaned_result)
display(df_cleaned)

,Austria_01,Austria_02,Austria_03,Austria_04,Austria_05,Austria_06,Austria_07,Croatia_01,Croatia_02,Croatia_03,...,Italy_06,MasterMix_,Norway_07,Norway_14,Swed_04,Swed_16,Swed_25,Swed_302,Swed_35,Swed_45
Calibrated m/z,,,,,,,,,,,,,,,,,,,,,
100.040385,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101.024427,0.0,6.741678e+06,2.785370e+06,0.0,0.0,0.0,0.0,0.0,8.520501e+06,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101.060776,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103.022346,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103.040016,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999.922333,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999.926835,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999.928320,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df= merge_and_export(files_toclean, df_cleaned, out_file)